<a href="https://colab.research.google.com/github/nayrr25/Aplicaciones-Financieras-e-IA/blob/main/BERT_examen3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fase 1: Importar las dependencias

In [1]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random

from google.colab import drive

In [2]:
!pip install bert-for-tf2
!pip install sentencepiece

In [3]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


# Fase 2: Pre procesado de datos

## Carga de los ficheros

Importamos los ficheros desde nuestro Google Drive personal.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import pandas as pd

# Asegúrate de usar la ruta correcta a tu archivo
file_path = '/content/drive/MyDrive/AplicacionesFinancieras/DataAnalyst.csv'

df = pd.read_csv(file_path)

In [6]:
X= df[['Job Title', 'Job Description','Sector']]
y= df[['Salary Estimate']]

In [7]:
X.head(6)

,Job Title,Job Description,Sector
0,"Data Analyst, Center on Immigration and Justic...",Are you eager to roll up your sleeves and harn...,Non-Profit
1,Quality Data Analyst,Overview\n\nProvides analytical and technical ...,Health Care
2,"Senior Data Analyst, Insights & Analytics Team...",We’re looking for a Senior Data Analyst who ha...,Information Technology
3,Data Analyst,Requisition NumberRR-0001939\nRemote:Yes\nWe c...,Information Technology
4,Reporting Data Analyst,ABOUT FANDUEL GROUP\n\nFanDuel Group is a worl...,"Arts, Entertainment & Recreation"
5,Data Analyst,About Cubist\nCubist Systematic Strategies is ...,Finance


## Preprocesar la información

### Limpieza

#### convertir el df en dataframe


In [8]:
df = pd.DataFrame(df)

# Limpiar el campo de estimación salarial
df["Salary Estimate"] = df["Salary Estimate"].str.replace(r" \(Glassdoor est.\)", "", regex=True)

In [9]:
# Ejemplo de normalización y conversión en variable numérica para la predicción
df["Salary Estimate"] = df["Salary Estimate"].str.replace('K', '000')


In [10]:
# conversión de X en data_clean, sumando cada uno de los vectores
data_clean= df['Job Title']+ " " + df['Job Description']+ " " + df['Sector']

In [11]:
df["data_clean"]= data_clean

In [12]:

# Convertir rangos a valores numéricos y calcular el promedio
df['min_salary'] = df['Salary Estimate'].str.split('-').str[0].str.replace('K', '').str.replace('$', '').astype(int) * 1000
df['max_salary'] = df['Salary Estimate'].str.split('-').str[1].str.replace('K', '').str.replace('$', '').astype(int) * 1000
df['avg_salary'] = (df['min_salary'] + df['max_salary']) / 2

# Mostrar las primeras filas después de la conversión para verificar
#print("Primeras filas después de la conversión:", df[['Salary Estimate', 'min_salary', 'max_salary', 'avg_salary']].head())


In [13]:
df['avg_salary']

0       51500000.0
1       51500000.0
2       51500000.0
3       51500000.0
4       51500000.0
          ...     
257    100000000.0
258    100000000.0
259    100000000.0
260    100000000.0
261    100000000.0
Name: avg_salary, Length: 262, dtype: float64

In [14]:
# de las tres posibilidades, uitilizaré el promedio

In [15]:
data_labels = df["avg_salary"].values
# Convert to a pandas Series
data_labels = pd.Series(data_labels)
pd.set_option('display.float_format', '{:.2f}'.format)
print(data_labels)

0      51500000.00
1      51500000.00
2      51500000.00
3      51500000.00
4      51500000.00
          ...     
257   100000000.00
258   100000000.00
259   100000000.00
260   100000000.00
261   100000000.00
Length: 262, dtype: float64


### Tokenization

Necesitaremos crear una capa BERT para tener acceso a los meta datos para el tokenizador (como el tamaño del vocabulario).

### Paso 1: Importar las bibliotecas necesarias
Asegúrate de que tienes las bibliotecas necesarias importadas para ejecutar el código, como tensorflow_hub y bert-for-tf2:

In [16]:
import tensorflow_hub as hub
import tensorflow as tf
#!pip install bert-for-tf2
import bert


### Paso 2: Cargar la Capa BERT desde TensorFlow Hub
El código carga un modelo BERT (bert_en_uncased_L-12_H-768_A-12) como una capa de Keras de TensorFlow Hub, que no será entrenable (trainable=False). Esto es útil para tareas que solo requieren la representación de BERT sin ajustar sus pesos:

In [17]:
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)

### Paso 3: Preparar el Tokenizer
Se prepara el tokenizer utilizando la información del modelo cargado:

Archivo de Vocabulario: vocab_file se obtiene directamente del modelo BERT cargado, que contiene el vocabulario usado durante el entrenamiento del modelo.
Case Sensitivity: do_lower_case indica si el modelo fue entrenado con textos en minúsculas, lo cual es importante para saber cómo procesar el texto de entrada.

In [18]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = bert.bert_tokenization.FullTokenizer(vocab_file, do_lower_case)

### Paso 4: Utilizar el Tokenizer
Una vez que tienes el tokenizer configurado, puedes usarlo para tokenizar tus textos. Por ejemplo:

In [19]:
text = "Here is an example sentence to tokenize using BERT tokenizer."
tokens = tokenizer.tokenize(text)
print(tokens)

['here', 'is', 'an', 'example', 'sentence', 'to', 'token', '##ize', 'using', 'bert', 'token', '##izer', '.']


### Paso 5: Aplicar el Tokenizer
Una vez que tienes la columna data_clean lista, puedes aplicar el tokenizer a cada entrada de esta columna. Usaremos una comprensión de lista para aplicar el tokenizer a cada fila del DataFrame.

In [20]:
# Suponiendo que 'tokenizer' ya está inicializado como mostraste anteriormente
df['tokens'] = df['data_clean'].apply(lambda x: tokenizer.tokenize(x))

In [21]:
df['tokens']

0      [data, analyst, ,, center, on, immigration, an...
1      [quality, data, analyst, overview, provides, a...
2      [senior, data, analyst, ,, insights, &, analyt...
3      [data, analyst, re, ##qui, ##sit, ##ion, numbe...
4      [reporting, data, analyst, about, fan, ##due, ...
                             ...                        
257    [data, analyst, -, qc, nes, ##co, resource, is...
258    [people, operations, &, data, analyst, job, de...
259    [lead, data, analyst, (, product, ), a, bit, a...
260    [data, analyst, -, iii, direct, client, requir...
261    [sql, data, analyst, job, title, :, senior, sq...
Name: tokens, Length: 262, dtype: object

Este código añadirá una nueva columna al DataFrame llamada 'tokens', donde cada entrada es la lista de tokens resultantes de aplicar el tokenizer BERT a la cadena correspondiente en 'data_clean'.

### Paso 6: Verificación
Es una buena práctica verificar que el proceso ha funcionado como se esperaba. Puedes inspeccionar las primeras filas del DataFrame para asegurarte de que los tokens se han generado correctamente:

In [22]:
print(df[['data_clean', 'tokens']].head())

                                          data_clean  \
0  Data Analyst, Center on Immigration and Justic...   
1  Quality Data Analyst Overview\n\nProvides anal...   
2  Senior Data Analyst, Insights & Analytics Team...   
3  Data Analyst Requisition NumberRR-0001939\nRem...   
4  Reporting Data Analyst ABOUT FANDUEL GROUP\n\n...   

                                              tokens  
0  [data, analyst, ,, center, on, immigration, an...  
1  [quality, data, analyst, overview, provides, a...  
2  [senior, data, analyst, ,, insights, &, analyt...  
3  [data, analyst, re, ##qui, ##sit, ##ion, numbe...  
4  [reporting, data, analyst, about, fan, ##due, ...  


### Paso 7: tokenizar las palabras

La función encode_sentence que mencionas está diseñada para convertir un texto de entrada en una lista de identificadores numéricos (IDs) correspondientes a los tokens en el vocabulario de BERT. Este proceso es una parte crucial en la preparación de datos para modelos basados en BERT, ya que estos modelos no trabajan directamente con texto plano sino con tokens que son convertidos en IDs numéricos.

In [23]:
def encode_sentence(sent):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sent))

### Paso 8: Conversión de Tokens a IDs
Una vez que el texto ha sido dividido en tokens, el siguiente paso es convertir cada token en su correspondiente ID numérico:

In [24]:
data_inputs = [encode_sentence(sentence) for sentence in data_clean]

Explicación de los Componentes
input_ids: Son los índices de los tokens en el vocabulario de BERT. Cada token del texto es convertido en un índice numérico que representa ese token en el vocabulario entrenado del modelo.

attention_mask: Es un arreglo que indica a modelo qué tokens deben ser atendidos y cuáles no. Por lo general, los tokens reales tienen una máscara de 1, y los tokens de relleno (padding) tienen una máscara de 0.

token_type_ids: Esta característica se utiliza principalmente para tareas que involucran dos textos distintos (como preguntas y respuestas). Indica a qué secuencia pertenece cada token. Para la mayoría de las aplicaciones de un solo texto, este campo no es necesario y puede estar ausente si el tokenizer no lo produce.

Opciones Adicionales
TensorFlow o PyTorch: El argumento return_tensors='pt' que usaste indica que los tensores deben ser devueltos como tensores de PyTorch. Si estás trabajando con TensorFlow, deberías cambiarlo a return_tensors='tf'.
Ejemplo Completo de Tokenización
Aquí te dejo un ejemplo completo que muestra cómo preparar los datos y tokenizarlos, asumiendo que data_clean es una lista de textos:

In [25]:
data_clean

0      Data Analyst, Center on Immigration and Justic...
1      Quality Data Analyst Overview\n\nProvides anal...
2      Senior Data Analyst, Insights & Analytics Team...
3      Data Analyst Requisition NumberRR-0001939\nRem...
4      Reporting Data Analyst ABOUT FANDUEL GROUP\n\n...
                             ...                        
257    Data Analyst - QC Nesco Resource is seeking a ...
258    People Operations & Data Analyst JOB DESCRIPTI...
259    Lead Data Analyst (Product) A BIT ABOUT OUR DA...
260    Data Analyst - III Direct Client Requirement\n...
261    SQL Data Analyst Job Title :Senior SQL Data An...
Length: 262, dtype: object

In [26]:
# Asumiendo que data_clean es una serie de pandas
texts = data_clean.tolist()

from transformers import BertTokenizer

# Cargar el tokenizer de BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenizar los textos
encoded_outputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

# Acceso a los componentes del resultado tokenizado
input_ids = encoded_outputs['input_ids']
attention_masks = encoded_outputs['attention_mask']
token_type_ids = encoded_outputs.get('token_type_ids')  # Puede ser None

# Verificación opcional: imprimir los primeros input_ids para verificar
print("Input IDs example:", input_ids[0])
print("Attention Mask example:", attention_masks[0])
if token_type_ids is not None:
    print("Token Type IDs example:", token_type_ids[0])



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Input IDs example: tensor([  101,  2951, 12941,  1010,  2415,  2006,  7521,  1998,  3425,  1006,
        25022,  3501,  1007,  2024,  2017,  9461,  2000,  4897,  2039,  2115,
        15114,  1998, 17445,  2951,  2000,  3298,  3343,  2689,  1029,  2079,
         2017,  5959,  9033, 26169,  2083,  3375,  2951, 13462,  2015,  2000,
         5665, 12717, 12556, 12878,  1998, 20062,  1029,  2079,  2017,  2156,
         4426,  2551,  2005,  1037,  5300,  1011,  5533,  3029,  2007,  1037,
         4432,  2000, 11147,  1996,  2087,  7827, 21321,  2015,  1997,  2256,
         2154,  1029,  2057,  2024,  2559,  2000, 10887,  1037,  4408,  1010,
         2524,  1011,  2551,  1010,  1998,  5541,  3265,  2007,  2844,  2951,
         2968,  4813,  1998,  1037,  7645,  8426,  2000, 11560,  1005,  1055,
         2916,  1012,  1996,  2951, 12941,  2097,  6509,  2007,  4106,  1998,
         7316,  3791,  2005, 12297,  2015,  2415,  2006,  7521,  1998,  3425,
         1006, 25022,  3501,  1007,  1010,  2

In [27]:
salaries= df["avg_salary"]

#Crear un Modelo de Predicción con BERT
Usaremos PyTorch como framework para construir un modelo de regresión que utilice BERT para procesar los textos y una capa lineal para predecir el salario. Primero, carga el modelo BERT preentrenado y añade una capa lineal para la predicción del salario:

In [28]:
import torch
from torch import nn
from transformers import BertModel

class SalaryPredictor(nn.Module):
    def __init__(self):
        super(SalaryPredictor, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.regressor = nn.Linear(768, 1)  # 768 es la dimensión de los embeddings de BERT

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output  # Salida agregada (CLS token)
        salary = self.regressor(pooled_output)
        return salary

# Inicializa el modelo
model = SalaryPredictor()


Paso 3: Preparar el DataLoader
Para entrenar el modelo, necesitas un DataLoader que combine los input_ids, attention_masks y los salarios. Suponiendo que ya has tokenizado data_clean y tienes encoded_outputs:

In [29]:
from torch.utils.data import TensorDataset, DataLoader

# Convertir los salarios a tensores de PyTorch
salary_tensors = torch.tensor(salaries.values, dtype=torch.float)

# Crear un TensorDataset
dataset = TensorDataset(encoded_outputs['input_ids'], encoded_outputs['attention_mask'], salary_tensors)

# Crear un DataLoader
data_loader = DataLoader(dataset, batch_size=16, shuffle=True)


Paso 4: Configurar el Entrenamiento
Define el optimizador y la función de pérdida, y procede a entrenar el modelo:

In [30]:
# from torch.optim import Adam

# optimizer = Adam(model.parameters(), lr=2e-5)
# criterion = nn.MSELoss()

# # Ciclo de entrenamiento
# model.train()
# for epoch in range(4):  # Número de epochs
#     for batch in data_loader:
#         input_ids, attention_mask, salaries = batch
#         optimizer.zero_grad()
#         predictions = model(input_ids, attention_mask)
#         loss = criterion(predictions.squeeze(), salaries)
#         loss.backward()
#         optimizer.step()
#     print(f'Epoch {epoch + 1}, Loss: {loss.item()}')


Paso 5: Evaluación del Modelo
Después de entrenar el modelo, evalúalo usando un conjunto de datos de prueba para verificar su rendimiento en la predicción de salarios:

In [31]:
# # Asegúrate de cambiar al modo de evaluación
# model.eval()
# # Implementa la evaluación aquí similar al bucle de entrenamiento, pero sin retropropagación


Este enfoque te proporciona una estructura completa desde la preparación de datos hasta el entrenamiento de un modelo que usa BERT para tareas de regresión, como predecir salarios basados en descripciones de trabajos. Ajusta los parámetros y la arquitectura según sea necesario para adaptarlos a tus necesidades específicas.

## Paso 4. 1. Optimización del Tamaño del Batch
Debido a las limitaciones de memoria y velocidad de la CPU, es posible que necesites ajustar el tamaño del batch para ser más pequeño. Esto reducirá la cantidad de memoria requerida por batch y evitará que el sistema se quede sin memoria durante el entrenamiento o la inferencia:

In [33]:
dataset

In [32]:
data_loader = DataLoader(dataset, batch_size=8, shuffle=True)  # Reducir el tamaño del batch si es necesario


## 4. 5. Gestión de Memoria
Asegúrate de gestionar bien la memoria:

Evita cargar grandes cantidades de datos en memoria a la vez.
Utiliza generadores o carga de datos por lotes cuando sea posible.
##. 4. 6. Uso Eficiente del DataLoader
Al utilizar el DataLoader, asegúrate de no sobrecargar la CPU. Puedes ajustar el número de workers a un número bajo para evitar el uso excesivo de múltiples núcleos:

In [36]:
data_loader = DataLoader(dataset, batch_size=8, shuffle=True, num_workers=1)  # Ajustar num_workers según la CPU

## 4. 2. Tiempo de Procesamiento
Prepárate para tiempos de procesamiento más largos. En una CPU, las operaciones no pueden ser paralelizadas al mismo nivel que en una GPU, por lo que cada paso de entrenamiento o inferencia puede tomar más tiempo.

## 4. 3. Uso de Modelos Pre-Optimizados
Considera usar versiones de BERT que están optimizadas para ser más eficientes. Por ejemplo, distilBERT es una versión más pequeña y más rápida de BERT que retiene la mayoría de su rendimiento:

In [37]:
from transformers import DistilBertModel, DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')


## 4.4. Uso de Modelos Cuantificados
La cuantificación reduce la precisión de los cálculos (de floating point a integers), lo que puede disminuir el uso de memoria y acelerar la inferencia. PyTorch ofrece soporte para cuantificación:

In [38]:
# Ejemplo de cuantificación dinámica para acelerar la inferencia
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)


# Modelo NUEVA CONFIGURACION
Paso 1: Preparar los Datos
Supongamos que ya tienes los datos tokenizados (input_ids, attention_mask) y una variable adicional como salaries.

Paso 2: Modificar la Arquitectura del Modelo
Para combinar estos datos, necesitas un modelo que no solo tome la salida de BERT, sino que también pueda integrar otras entradas. Aquí hay un ejemplo de cómo puedes estructurar este modelo:

In [39]:
import torch
from torch import nn
from transformers import BertModel

class BertWithAdditionalFeatures(nn.Module):
    def __init__(self, num_additional_features):
        super(BertWithAdditionalFeatures, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.regressor = nn.Linear(768 + num_additional_features, 1)  # Asume 768 dimensiones de BERT + tus características adicionales

    def forward(self, input_ids, attention_mask, additional_features):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        # Concatenar la salida de BERT con las características adicionales
        combined_features = torch.cat((pooled_output, additional_features), 1)
        return self.regressor(combined_features)

# Inicializa el modelo con el número de características adicionales, ej: 1 si es solo el salario
model = BertWithAdditionalFeatures(num_additional_features=1)


Paso 3: Preparar las Características Adicionales
Asegúrate de que las características adicionales estén en el formato adecuado, por ejemplo, un tensor de PyTorch:

In [40]:
# Asegúrate de que los salarios son un tensor con la forma correcta
additional_features = torch.tensor(salaries.values, dtype=torch.float32).view(-1, 1)  # Redimensiona si es necesario


Paso 4: Crear TensorDataset y DataLoader
Cuando crees el TensorDataset y DataLoader, incluye estas características adicionales:

In [41]:
from torch.utils.data import TensorDataset, DataLoader

# Suponiendo que encoded_outputs ya están definidos como antes
input_ids = encoded_outputs['input_ids']
attention_mask = encoded_outputs['attention_mask']

dataset = TensorDataset(input_ids, attention_mask, additional_features, salary_tensors)
data_loader = DataLoader(dataset, batch_size=16, shuffle=True)


Paso 5: Entrenamiento del Modelo
Cuando entrenes el modelo, asegúrate de pasar todas las entradas necesarias:

In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader
from transformers import BertModel

optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
criterion = nn.MSELoss()
num_epochs = 1  # Puedes ajustar esto según tus necesidades y recursos


model.train()
for epoch in range(num_epochs):
    for batch in data_loader:
        input_ids, attention_mask, additional_features, salaries = batch
        optimizer.zero_grad()
        predictions = model(input_ids, attention_mask, additional_features)
        loss = criterion(predictions.squeeze(), salaries)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch + 1}, Loss: {loss.item()}')
